# Bibliotecas

Importação de bibliotecas necessarias para o projeto

In [1]:
!pip install boto3 --quiet
!pip install sagemaker --quiet
!pip install spacy --quiet
!python -m spacy download pt_core_news_sm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 71.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd
import boto3
import sagemaker

# Stopworrds
import nltk
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from collections import Counter
nltk.download('punkt')

import spacy
nlp = spacy.load('pt_core_news_sm')

# Treino e Teste
from sklearn.model_selection import train_test_split

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Conexão S3

Conexão no s3 para verificar região e role

In [3]:
ACCESS_ID = ""
ACCESS_KEY = ""
region = ""

In [4]:
# abrir sessão
session = boto3.Session(aws_access_key_id=ACCESS_ID, aws_secret_access_key= ACCESS_KEY)

In [5]:
boto_session = boto3.session.Session(aws_access_key_id=ACCESS_ID, aws_secret_access_key= ACCESS_KEY)
region = boto_session.region_name
print(region)

None


## Funções

Função de lematização

In [6]:
# função de lematização completa do documento
def lemmatizer_text(text):
    doc = nlp(text, disable=["parser", "ner"])
    return " ".join(token.lemma_ for token in doc)

# Importação DataFrame

Importação de dataframe importado no S3

In [7]:
# Configurar a conexão com o S3
s3 = boto3.client('s3', aws_access_key_id=ACCESS_ID, aws_secret_access_key= ACCESS_KEY)
bucket_name = 'vitor-fiap'
file_key = 'dataset_nlp/tickets_reclamacoes_classificados.csv'

# Carregar dataset
response = s3.get_object(Bucket=bucket_name, Key=file_key)
df = pd.read_csv(response['Body'], delimiter=';')
df

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."
...,...,...,...,...
21067,3094545,2018-12-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Depois de ser um cliente de cartão de persegui...
21068,3091984,2018-12-05T12:00:00-05:00,Roubo / Relatório de disputa,"Na quarta -feira, xx/xx/xxxx, liguei para o Ch..."
21069,3133355,2019-01-25T12:00:00-05:00,Roubo / Relatório de disputa,Não estou familiarizado com o XXXX Pay e não e...
21070,3110963,2018-12-27T12:00:00-05:00,Outros,Eu tive crédito impecável por 30 anos. Eu tive...


# Feature Engineering

Deletando colunas que não serão necessarias para o processo de NLP, sendo elas id_reclamacao e data_abertura.

In [8]:
df.drop(columns=['id_reclamacao', 'data_abertura'], inplace=True)

Tratamento da coluna descricao_reclamacao para termos um melhor resultado

In [9]:
# Trasnformando descricao_reclamacao em lower case
df.descricao_reclamacao = df.descricao_reclamacao.str.lower()

# Dicionario de dubstituições
replace_dict = {'xx': ''}

# Substituindo os valores XX e 00 por vazio
df.descricao_reclamacao = df.descricao_reclamacao.replace(replace_dict, regex=True)
df

,categoria,descricao_reclamacao
0,Hipotecas / Empréstimos,"bom dia, meu nome é e agradeço se você puder..."
1,Cartão de crédito / Cartão pré-pago,atualizei meu cartão em //2018 e fui informa...
2,Cartão de crédito / Cartão pré-pago,o cartão chase foi relatado em //2019. no enta...
3,Cartão de crédito / Cartão pré-pago,"em //2018, enquanto tentava reservar um ticket..."
4,Serviços de conta bancária,"meu neto me dê cheque por {$ 1600,00} eu depos..."
...,...,...
21067,Cartão de crédito / Cartão pré-pago,depois de ser um cliente de cartão de persegui...
21068,Roubo / Relatório de disputa,"na quarta -feira, //, liguei para o chas, meu ..."
21069,Roubo / Relatório de disputa,não estou familiarizado com o pay e não enten...
21070,Outros,eu tive crédito impecável por 30 anos. eu tive...


## Stop Words

Criação de variaveis para stop words

In [10]:
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stops_nlp = list(nlp.Defaults.stop_words)
stops_comb = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))
print('Stops_nltk: ', len(stops_nltk))
print('Stops_nlp:  ', len(stops_nlp))
print('Stops_comb: ', len(stops_comb))

Stops_nltk:  207
Stops_nlp:   416
Stops_comb:  500


Lematização da coluna descricao_reclamacao

In [11]:
# aplica a lematização no dataframe e criando nova coluna
df['descricao_reclamacao_lemma'] = df.descricao_reclamacao.apply(lemmatizer_text)
df

,categoria,descricao_reclamacao,descricao_reclamacao_lemma
0,Hipotecas / Empréstimos,"bom dia, meu nome é e agradeço se você puder...","bom dia , meu nome ser e agradeço se você p..."
1,Cartão de crédito / Cartão pré-pago,atualizei meu cartão em //2018 e fui informa...,atualizei meu cartão em //2018 e ser inform...
2,Cartão de crédito / Cartão pré-pago,o cartão chase foi relatado em //2019. no enta...,o cartão chase ser relatar em //2019 . em o en...
3,Cartão de crédito / Cartão pré-pago,"em //2018, enquanto tentava reservar um ticket...","em //2018 , enquanto tentar reservar um ticket..."
4,Serviços de conta bancária,"meu neto me dê cheque por {$ 1600,00} eu depos...","meu neto eu dê cheque por { $ 1600,00 } eu dep..."
...,...,...,...
21067,Cartão de crédito / Cartão pré-pago,depois de ser um cliente de cartão de persegui...,depois de ser um cliente de cartão de persegui...
21068,Roubo / Relatório de disputa,"na quarta -feira, //, liguei para o chas, meu ...","em o quarto -feira , // , liguei para o cha , ..."
21069,Roubo / Relatório de disputa,não estou familiarizado com o pay e não enten...,não estar familiarizado com o pay e não ente...
21070,Outros,eu tive crédito impecável por 30 anos. eu tive...,eu ter crédito impecável por 30 ano . eu ter c...


Deletando a coluna descricao_reclamacao

In [12]:
df.drop(columns=['descricao_reclamacao'], inplace=True)

# Treino e Teste

Separação de treino e teste

In [13]:
df_train, df_test = train_test_split(df, test_size = 0.25, random_state = 42)

# Exportação

Exportação dos dataframes de treino e teste para rodar no Amazon Comprehend

In [14]:
df_train.to_csv("tickets_reclamacoes_classificados_treino.csv", index=False, header=False)

In [15]:
df_test.to_csv("tickets_reclamacoes_classificados_teste.csv", index=False, header=False)

Deletando a coluna categoria do dataframe df_test para exportar apenas o documento e rodar o arquivo "tickets_reclamacoes_classificados_teste2.csv" como job do modelo treinado

In [16]:
df_test.drop(columns=['categoria'], inplace=True)

In [17]:
df_test.to_csv("tickets_reclamacoes_classificados_teste2.csv", index=False, header=False)